In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

In [10]:
week_day_Jan_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

In [11]:
link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in week_day_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['PM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jan_list)

63360

In [13]:
x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2640)

In [15]:
x[:,:2]

matrix([[ 340343.68736559,  344580.87522118],
        [ 354547.26763359,  353278.70784122],
        [ 641711.07035668,  640384.03128932],
        [ 656881.3361256 ,  655683.87107291],
        [ 172686.99226455,  168656.1190212 ],
        [ 204528.68874518,  203082.75778331],
        [ 398010.66188246,  395633.44759811],
        [ 422822.4764407 ,  415113.09819685],
        [ 383014.54220228,  382856.1441145 ],
        [ 391940.6895835 ,  394702.01014943],
        [ 357197.25720961,  381656.98946946],
        [ 326764.44115381,  326736.11874513],
        [  97105.25740513,  100224.41504694],
        [  97581.89869044,  101715.94476647],
        [ 151862.83322844,  147102.13551821],
        [ 142779.49645596,  142513.79959996],
        [ 211467.79924803,  210514.19810158],
        [ 216628.85539941,  223073.95679241],
        [ 370758.99324132,  367564.39726109],
        [ 357073.81439455,  364699.19647178],
        [ 559036.00914374,  535975.82065556],
        [ 527643.68599091,  529487

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e-01, 3e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.72806577e+07  0.00000000e+00  4.24e+03 2.04e+03  1.00e+06     0s
   1  -1.55789223e+07 -6.61241552e+06  1.27e+03 6.11e+02  3.37e+05     0s
   2  -1.32302022e+07 -1.52641649e+07  1.05e+02 5.06e+01  8.65e+04     0s
   3  -1.36774248e+07 -1.39940392e+07  1.05e-04 5.06e-05  5.6

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jan_weekday_PM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[3.060168115063129e-09,
 358892.66030953,
 12268.115863617066,
 26457.497844365007,
 108234.39767681227,
 4.861286829328294e-06,
 4.878368759741211e-06,
 3.675581250032163e-09,
 1.8949877512097417e-10,
 375419.8211931551,
 1.8949877482458874e-10,
 1.89498775081114e-10,
 1.894913885358131e-10,
 1.907687820820807e-10,
 381661.88520501886,
 1.2691486892239202e-10,
 0.8928472360601774,
 31814.928589829702,
 152808.01587198453,
 521294.8545858985,
 4.863078767148102e-06,
 67960.82406163705,
 306779.2617902605,
 61982.14020426574,
 0.00032673252538868713,
 0.00032672647469427635,
 2.746580772448339e-06,
 176462.12298909973,
 0.6451872045199707,
 1.269148688807683e-10,
 246551.82638791358,
 20427.906961365446,
 173618.73977246197,
 3.430062658854113e-08,
 2.7387180993823797e-06,
 388682.0439100337,
 1.2691486892407947e-10,
 8.391564101651737e-06,
 20220.287548111868,
 60857.23660231244,
 724940.6930788545,
 51098.21923416373,
 1.9783183226383064e-05,
 1.269140547431115e-10,
 5.779211488136709